<a href="https://colab.research.google.com/github/Kiana-M/Refreshers-and-Tutorials/blob/main/OOP_review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Basics of Object Oriented Programming

### **Classes and Objects**

**Class**: A blueprint for creating objects (instances). It defines the attributes (data) and methods (functions) that the objects will have.

**Object**: An instance of a class. When you create an object, you are making an instance of the class.

### **Attributes and Methods:**

**Attributes**: Variables that belong to an object or class (also known as properties).

**Methods**: Functions defined inside a class that describe the behaviors of an object.


### **Encapsulation:**

The practice of keeping an object's internal state private and only exposing it through methods. This protects the integrity of the data.

### **Inheritance:**

A way to create a new class using an existing class. The new class (child) inherits the attributes and methods of the existing class (parent), but can also have additional features.

### **Polymorphism:**

Allows objects of different types to be treated as if they are objects of a common parent class. Different classes can define the same method and behave differently when that method is called.

### **Abstraction**:

Hides complex implementation details and shows only the necessary features of an object.

# Toy Project Overview: Banking System

We'll create classes for a `BankAccount` system that includes:
- **BankAccount (abstract class)**: Defines the basic structure for all account types.
- **CheckingAccount** and **SavingsAccount** (inherited classes): Each with unique behavior.
- **Transaction**: Represents deposits, withdrawals, and transfers between accounts.

### Banking System Design

1. **BankAccount** (abstract class) — represents the blueprint of an account with:
   - **Balance**: A protected attribute.
   - **Deposit** and **Withdraw** methods, to be implemented in child classes.
   
2. **CheckingAccount** and **SavingsAccount** (concrete classes) — represent specific types of accounts:
   - CheckingAccount: No interest; includes overdraft protection.
   - SavingsAccount: Earns interest; allows only a limited number of withdrawals.

3. **Transaction** class — simulates deposit, withdrawal, and transfer between accounts.



In [2]:
from abc import ABC, abstractmethod

# 1. Abstract BankAccount class
class BankAccount(ABC):
    def __init__(self, owner, balance=0):
        self.owner = owner
        self._balance = balance  # Protected attribute for balance

    @property
    def balance(self):
        return self._balance

    @abstractmethod
    def deposit(self, amount):
        pass

    @abstractmethod
    def withdraw(self, amount):
        pass

# 2. CheckingAccount class with overdraft protection
class CheckingAccount(BankAccount):
    def __init__(self, owner, balance=0, overdraft_limit=100):
        super().__init__(owner, balance)
        self.overdraft_limit = overdraft_limit

    def deposit(self, amount):
        if amount > 0:
            self._balance += amount
            return f"{amount} deposited. New balance: {self._balance}"
        return "Deposit amount must be positive."

    def withdraw(self, amount):
        if 0 < amount <= (self._balance + self.overdraft_limit):
            self._balance -= amount
            return f"{amount} withdrawn. New balance: {self._balance}"
        return f"Insufficient funds. Overdraft limit is {self.overdraft_limit}."

# 3. SavingsAccount class with interest and withdrawal limits
class SavingsAccount(BankAccount):
    def __init__(self, owner, balance=0, interest_rate=0.02, withdrawal_limit=3):
        super().__init__(owner, balance)
        self.interest_rate = interest_rate
        self.withdrawals = 0
        self.withdrawal_limit = withdrawal_limit

    def deposit(self, amount):
        if amount > 0:
            self._balance += amount
            return f"{amount} deposited. New balance: {self._balance}"
        return "Deposit amount must be positive."

    def withdraw(self, amount):
        if self.withdrawals < self.withdrawal_limit and amount <= self._balance:
            self._balance -= amount
            self.withdrawals += 1
            return f"{amount} withdrawn. New balance: {self._balance}"
        elif self.withdrawals >= self.withdrawal_limit:
            return "Withdrawal limit reached for this period."
        return "Insufficient funds."

    def add_interest(self):
        interest = self._balance * self.interest_rate
        self._balance += interest
        return f"Interest added. New balance: {self._balance}"

# 4. Transaction class for deposits, withdrawals, and transfers
class Transaction:
    @staticmethod
    def deposit(account, amount):
        return account.deposit(amount)

    @staticmethod
    def withdraw(account, amount):
        return account.withdraw(amount)

    @staticmethod
    def transfer(from_account, to_account, amount):
        withdraw_message = from_account.withdraw(amount)
        if "withdrawn" in withdraw_message:
            deposit_message = to_account.deposit(amount)
            return f"Transfer successful: {withdraw_message} | {deposit_message}"
        return "Transfer failed: insufficient funds."

# Example Usage
if __name__ == "__main__":
    # Create accounts
    checking = CheckingAccount("Alice", balance=500)
    savings = SavingsAccount("Alice", balance=1000)

    # Perform transactions
    print(Transaction.deposit(checking, 200))           # Depositing in checking
    print(Transaction.withdraw(savings, 150))           # Withdrawing from savings
    print(Transaction.transfer(checking, savings, 100)) # Transfer between accounts
    print(savings.add_interest())                       # Adding interest to savings
    print(Transaction.withdraw(savings, 900))           # Withdraw beyond balance in savings
    print(Transaction.withdraw(checking, 700))          # Withdraw within overdraft limit


200 deposited. New balance: 700
150 withdrawn. New balance: 850
Transfer successful: 100 withdrawn. New balance: 600 | 100 deposited. New balance: 950
Interest added. New balance: 969.0
900 withdrawn. New balance: 69.0
700 withdrawn. New balance: -100


### Key OOP Concepts in the Project

1. **Abstraction**:
   - The `BankAccount` class is abstract, meaning it defines a template for accounts without implementing specifics.
   - `deposit` and `withdraw` are abstract methods, forcing subclasses to define them.

2. **Encapsulation**:
   - `_balance` is a protected attribute (denoted by the underscore) and accessed through methods, which is good practice for protecting data.

3. **Inheritance**:
   - `CheckingAccount` and `SavingsAccount` inherit from `BankAccount` and extend its functionality.

4. **Polymorphism**:
   - Both `CheckingAccount` and `SavingsAccount` implement the `deposit` and `withdraw` methods, but each behaves differently based on the account type.

5. **Composition**:
   - `Transaction` uses `BankAccount` objects to perform operations, demonstrating composition where `Transaction` is not an account but can interact with accounts.

This project covers all essential OOP concepts and provides a realistic example of designing and implementing a small but practical application in Python!.

# More on Encapulation

In Python, the `@property` decorator allows you to define methods that act like attributes. With `@property`, you can control how an attribute’s value is accessed or modified without directly exposing it, enhancing **encapsulation**.

### How `@property` Works

1. **Getter**: When you use `@property` on a method, it allows you to access that method like an attribute.
2. **Setter (Optional)**: You can define a `@property_name.setter` method to control how the attribute can be set.

### Why Use `@property`?

The main advantages are:
- **Encapsulation**: You can expose an attribute but control its access.
- **Readability**: `@property` lets you keep the attribute-like syntax even when there’s logic behind accessing the value.
- **Data Validation**: You can add validation or transformation logic in the getter and setter.

### Example

Let’s revisit a `BankAccount` example and use `@property` for the balance attribute:

```python
class BankAccount:
    def __init__(self, owner, balance=0):
        self.owner = owner
        self._balance = balance  # Internal (protected) variable for balance

    @property
    def balance(self):
        # Getter: allows reading the balance as if it's an attribute
        return self._balance

    @balance.setter
    def balance(self, value):
        # Setter: allows modifying the balance with some validation
        if value >= 0:
            self._balance = value
        else:
            raise ValueError("Balance cannot be negative.")

# Example usage
account = BankAccount("Alice", 100)
print(account.balance)  # Accesses balance using the getter

account.balance = 200   # Modifies balance using the setter
print(account.balance)   # Updated balance

# Uncommenting the line below would raise an error due to the validation in the setter
# account.balance = -50
```

Here:
- `@property` makes `balance` accessible as an attribute rather than a method.
- The setter (`@balance.setter`) validates the value before updating the `_balance` attribute.

This is especially useful for managing data access, ensuring your code is robust and easy to maintain.

# Some Q&A with ChatGPT, Republic style:

https://chatgpt.com/share/671bbf09-6394-8013-9e92-18e888884296

**Q**: